In [1]:
from first_stage_frlm import first_stage_frlm
from second_stage_frlm import second_stage_frlm
from first_stage_frlm import get_minimal_subsets
import networkx as nx
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import random
import itertools
import pulp

In [2]:
import pickle

G=pickle.load( open('data/network_digital_twin_v0.2.pickle','rb'))

In [3]:
#Get data from df
df_links = nx.to_pandas_edgelist(G)
df_nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [4]:
df_links.loc[df_links.target == "8866426"]

,source,target,lat_p50,nap_stdDev,nap_p10,nap_p50,SeaFairingLength,Classification,lat_count,length_deg,...,lat_p95,length_m,StartJunctionId,GeoType,PushedLength,lat_p90,lat_p5,PushedWidth,Wkt,GeneralWidth
232,8867817,8866426,NaN,NaN,NaN,NaN,NaN,CEMT,0,NaN,...,NaN,426.400769,8867817,section,NaN,NaN,NaN,NaN,"LINESTRING (4.28216894090561 51.7207661398166,...",NaN


In [5]:
df_links.loc[df_links.Name=='Vaarwegvak van 0 tot 0 - H'].dropna(how='all')
# df_links.loc[df_links.target == "8866426"]
# df_nodes.loc[df_nodes.index=="30534997"].dropna(axais=1)

,source,target,lat_p50,nap_stdDev,nap_p10,nap_p50,SeaFairingLength,Classification,lat_count,length_deg,...,lat_p95,length_m,StartJunctionId,GeoType,PushedLength,lat_p90,lat_p5,PushedWidth,Wkt,GeneralWidth
8,8861581,8863189,NaN,NaN,NaN,NaN,NaN,CEMT,0,NaN,...,NaN,613.104746,8861581,section,NaN,NaN,NaN,NaN,"LINESTRING (6.05213774082153 53.1160238634629,...",NaN
10,8860954,8864050,NaN,NaN,NaN,NaN,NaN,CEMT,0,NaN,...,NaN,1234.583805,8860954,section,NaN,NaN,NaN,NaN,"LINESTRING (4.44741233930644 51.8818778559588,...",NaN
15,8865802,8862540,NaN,NaN,NaN,NaN,NaN,CEMT,0,NaN,...,NaN,576.337751,8865802,section,NaN,NaN,NaN,NaN,"LINESTRING (4.4144439627459 51.9093040799995, ...",NaN
16,8861605,8863756,NaN,NaN,NaN,NaN,NaN,CEMT,0,NaN,...,NaN,271.985423,8861605,section,NaN,NaN,NaN,NaN,"LINESTRING (4.4130241878863 51.9111080613344, ...",NaN
22,8864595,8864137,NaN,NaN,NaN,NaN,175.0,CEMT,0,NaN,...,NaN,116.684543,8864137,section,NaN,NaN,NaN,NaN,"LINESTRING (4.63430154239131 51.7944041770278,...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14479,Berth20,Berth22,NaN,0.192602,0.968794,1.380204,NaN,CEMT,0,NaN,...,NaN,40.250350,Berth20,section,NaN,NaN,NaN,NaN,LINESTRING (6.090474005761066 51.8569340528272...,NaN
14480,Berth21,8860919,NaN,0.120221,1.342185,1.541399,NaN,CEMT,0,NaN,...,NaN,85.777056,Berth21,section,NaN,NaN,NaN,NaN,LINESTRING (6.090733700735698 51.8572503422915...,NaN
14481,Berth21,Berth22,NaN,0.025505,1.347982,1.366488,NaN,CEMT,0,NaN,...,NaN,5.033140,Berth22,section,NaN,NaN,NaN,NaN,LINESTRING (6.090704836324389 51.8572151875520...,NaN
14487,Berth39,Berth40,NaN,0.109707,-6.139389,-6.017772,115.0,CEMT,0,NaN,...,NaN,133.630830,Berth39,section,NaN,NaN,NaN,NaN,"LINESTRING (3.82593469906891 51.3222249113116,...",16.0


In [6]:
a=df_nodes.index

In [7]:
berths = []
for harbour in a:
    if "Berth" in harbour:
        berths.append(harbour)

## Random origin and destination picked using:

https://deploy-preview-19–digitwin-waterways.netlify.app/#/

(8860852, B43966_B, 10) (8860852, B27067_B, 5)

Now check if it works with two scripts!

In [ ]:
# scripts clearly still need to be adjusted to new network slightly

In [8]:
df_b, df_g, df_eq_fq = first_stage_frlm(G, 10*(10^3), [("8860852", "8862426", 10)])

{('8860852', '8862426'): 16159.45677737489}


In [9]:
df_b

""
q
"(8860852, 8862426)"


In [10]:
locations = second_stage_frlm(10, 20, df_g, df_b, df_eq_fq)

KeyError: 0